In [168]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *

In [169]:
rules = pd.read_csv(config.files['service_rules'])

In [170]:
class PipedRegexes:
    
    def __init__(self, matching_param, prefix, suffix):
        self.param = matching_param
        self.prefix = prefix
        self.suffix = suffix
        
    def canHandle(self, param):
        return self.param == param
        
    def join(self, groupby):
        return (
            self.prefix
            + groupby
            + self.suffix
        )
    


In [171]:
regexes = PipedRegexes('M', 'prefix', 'suffix')
print(regexes.)

AttributeError: 'RegexDecorator' object has no attribute 'prefix'